Downloading Data:
=================

- Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
- Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [13]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-01-22 14:52:30
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [41]:
# Setup the environment with pip installs and imports

installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/43.6 MB ? eta -:--:--
    --------------------------------------- 1.0/43.6 MB 3.3 MB/s eta 0:00:13
   -- ------------------------------------- 2.4/43.6 MB 5.0 MB/s eta 0:00:09
   --- ------------------------------------ 3.9/43.6 MB 5.7 MB/s eta 0:00:07
   ----- ---------------------------------- 5.8/43.6 MB 6.5 MB/s eta 0:00:06
   ------ --------------------------------- 7.6/43.6 MB 7.1 MB/s eta 0:00:06
   -------- ------------------------------- 9.4/43.6 MB 7.2 MB/s eta 0:00:05
   ---------- ----------------------------- 11.3/43.6 MB 7.6 MB/s eta 0:00:05
   ------------ --------------------------- 13.1/43.6 MB 7.8 MB/s eta 0:00:04
   ------------- -------------------------- 14.9/43.6 MB 7.9 MB/s eta 0:00:04
   --------------- ------------------------ 17.0/43.6 MB 8.1 MB/s eta 0:00:04
   ----------------- ---------------------- 18.9/43.6 MB 8.2 MB/s eta 0:00:04
   -----

In [14]:


# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)


In [31]:
# Your client credentials
#Client ID: sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9
#Client secret: BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5
client_id = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

In [33]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [73]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [112]:
from PIL import Image
from io import BytesIO

oauth = SetupOAuth(client_id, client_secret)

maxlat = 50.8
minlat = 50.7
minlon = -1.4
maxlon = -1.3
bboxstring = CreateBBoxString(minlon, minlat, maxlon, maxlat)

print(bboxstring)


date = "2024-11-26"
time_range = CreateTimerangeString(datetime.datetime.strptime(date, "%Y-%m-%d"))

evalscript_clean = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "SCL"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

evalscript_cloud = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "SCL"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
  if ([8, 9, 10].includes(sample.SCL)) {
    return [1, 0, 1]
  } else {
    return [0, 0, 0]
  }
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
            "bbox": [-1.4, 50.7, -1.3, 50.8],
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {"timeRange": time_range}
            }
        ]
    },
    "output": {
        "width":  1024,
        "height": 1024,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/png"}
            }
        ]
    },
    "evalscript": evalscript_clean
}












evalscript_GM = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04/10000, 2.5 * sample.B03/10000, 2.5 * sample.B02/10000];
}
"""

request_GM = {
  "input": {
    "bounds": {
      "bbox": [-10, 50, 0, 60]
    },
    "data": [
      {
        "dataFilter": {
          "timeRange": {
            "from": "2023-04-01T00:00:00Z",
            "to": "2023-04-01T23:59:59Z"
          }
        },
        "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

      }
    ]
  },
  "output": {
    "width": 1024,
    "height": 1024,
    "responses": [
      {
        "identifier": "default",
        "format": {
          "type": "image/jpeg"
        }
      }
    ]
  },
  "evalscript": evalscript_GM,
}













url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request_GM)

# Output the result to a file
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    image.save("output_image6.png")
else:
    print(f"Request failed with status code {response.status_code}")



[-1.4, 50.7, -1.3, 50.8]


Design

- We have an "input stripe" of data, with min/max lat long values.
- Pass across the input stripe at a 0.1 degree level.
    - Lower res while testing
- Download cloud image
- Download "clean" unaltered image
    - Fix the clean image exposure


In [ ]:
stripe_min_lat = 50.7
stripe_max_lat = 50.8
stripe_min_lon = -1.4
stripe_max_lon = -1.3

In [113]:
from PIL import Image, ImageEnhance
import numpy as np


def enhance_image_brightness(input_path, output_path, desired_brightness):
    # Load the uploaded image
    image = Image.open(input_path)

    # Convert the image to grayscale and calculate its brightness
    grayscale_image = image.convert("L")
    brightness = np.mean(np.array(grayscale_image))

    # Calculate enhancement factor
    enhancement_factor = desired_brightness / brightness

    # Enhance the image brightness
    enhancer = ImageEnhance.Brightness(image)
    enhanced_image = enhancer.enhance(enhancement_factor)

    # Save the enhanced image
    enhanced_image.save(output_path)

    #return brightness, output_path

# Example usage
desired_brightness = 100
input_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image6.png'
output_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image6_b.png'
enhance_image_brightness(input_path, output_path, desired_brightness)


In [94]:
import numpy as np
import cv2

def analyze_image3(input_path, output_path, max_distance=100):

    # Load the image
    image = cv2.imread(input_path)

    # Define magenta color in BGR
    magenta = np.array([255, 0, 255])

    max_value = 255

    # Factor to multiply the distance by to normalize it to the output greyscale range [0, 255]
    max_distance_to_max_value = max_value / max_distance

    # Create an output image initialized to black
    output_image = np.zeros(image.shape[:2], dtype=np.float32)
    output_image.fill(max_value)

    # Iterate through each pixel in the image
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):

            if np.array_equal(image[y, x], magenta):

                # Write a pattern of values into the output image
                for dy in range(-max_distance, max_distance + 1):
                    for dx in range(-max_distance, max_distance + 1):

                        # Find the output pixel coordinates and check its within the image bounds
                        ny, nx = y + dy, x + dx
                        if 0 <= ny < image.shape[0] and 0 <= nx < image.shape[1]:

                            # Calculate the distance between the current pixel and the magenta pixel
                            distance = np.sqrt(dx**2 + dy**2)

                            # Clip the distance to the maximum distance, so all potential values are [0, max_distance]
                            distance = min(distance, max_distance)

                            # Normalize the distance to the range [0, 255]
                            val = distance * max_distance_to_max_value

                            # The image accumulates the minimum distance value to a magenta pixel,
                            # either the current value or the new value
                            output_image[ny, nx] = min(output_image[ny, nx], distance)

    # Normalize the output image to the range [0, 255]
    output_image = (output_image).astype(np.uint8)

    # Save the output image
    cv2.imwrite(output_path, output_image)


In [96]:
import numpy as np
import cv2

def analyze_image_optimized(input_path, output_path, max_distance=100):
    # Load the image
    image = cv2.imread(input_path)

    # Define magenta color in BGR
    magenta = np.array([255, 0, 255])

    # Create a mask of magenta pixels
    mask = np.all(image == magenta, axis=-1).astype(np.uint8)

    # Compute the Euclidean distance transform for the mask
    distances = cv2.distanceTransform(1 - mask, cv2.DIST_L2, 3)

    # Clip distances to max_distance and normalize to [0, 255]
    distances = np.clip(distances, 0, max_distance)
    distances = (distances / max_distance * 255).astype(np.uint8)

    # Save the output image
    cv2.imwrite(output_path, distances)


In [97]:

input_cloud_path  = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image4.png'
output_cloud_path = 'C:/Util/GlobeLibrary_MapPrep/SupportingTools/MapDownload/output_image4_g.png'

# Example usage
analyze_image_optimized(input_cloud_path, output_cloud_path)

In [ ]:
Path - Code, bl, size



RootDir = "C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib"
FilePath = "/Lvl1_5x5/EL/Sat_EL_DD.png"




EL_DD, -50, 165, 5

